## Implementing a ChatGPT App with LangChain from Scratch



In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

OPENAI_API_KEY="sk-proj-*************"
PINECONE_API_KEY = "*******"


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1, openai_api_key = OPENAI_API_KEY)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#         SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in German.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Your prompt:  Hi




> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in German.
Human: Hi

> Finished chain.
{'content': 'Hi', 'text': 'Hallo! Wie kann ich Ihnen helfen?'}
--------------------------------------------------


Your prompt:  how are you?




> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in German.
Human: how are you?

> Finished chain.
{'content': 'how are you?', 'text': 'Mir geht es gut, danke! Wie kann ich Ihnen helfen?'}
--------------------------------------------------


...



> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in German.
Human: my name is noordeen

> Finished chain.
{'content': 'my name is noordeen', 'text': 'Guten Tag, Noordeen! Wie kann ich Ihnen helfen?'}
--------------------------------------------------


Your prompt:  what is my name




> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in German.
Human: what is my name

> Finished chain.
{'content': 'what is my name', 'text': 'Entschuldigung, aber ich kenne nicht deinen Namen. Wie kann ich dir heute helfen?'}
--------------------------------------------------


Your prompt:  bye


Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [5]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1, openai_api_key = OPENAI_API_KEY)

# 2. Create memory 
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  hi


{'content': 'hi', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How are you doing today?')], 'text': 'Hello! How are you doing today?'}
--------------------------------------------------


Your prompt:  my name is noordeen


{'content': 'my name is noordeen', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How are you doing today?'), HumanMessage(content='my name is noordeen'), AIMessage(content='Nice to meet you, Noordeen! How can I assist you today?')], 'text': 'Nice to meet you, Noordeen! How can I assist you today?'}
--------------------------------------------------


Your prompt:  what is my name?


{'content': 'what is my name?', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How are you doing today?'), HumanMessage(content='my name is noordeen'), AIMessage(content='Nice to meet you, Noordeen! How can I assist you today?'), HumanMessage(content='what is my name?'), AIMessage(content='Your name is Noordeen. How can I help you today, Noordeen?')], 'text': 'Your name is Noordeen. How can I help you today, Noordeen?'}
--------------------------------------------------


Your prompt:  bye


Goodbye!


## Saving Chat Sessions

In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1, openai_api_key = OPENAI_API_KEY)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  hi


{'content': 'hi', 'chat_history': [], 'text': 'Hello! How are you doing today?'}
--------------------------------------------------


Your prompt:  My name is noordeen


{'content': 'My name is noordeen', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How are you doing today?')], 'text': 'Nice to meet you, Noordeen! How can I assist you today?'}
--------------------------------------------------


Your prompt:  what is my name?


{'content': 'what is my name?', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How are you doing today?'), HumanMessage(content='My name is noordeen'), AIMessage(content='Nice to meet you, Noordeen! How can I assist you today?')], 'text': 'Your name is Noordeen. Is there anything else you would like to know or talk about?'}
--------------------------------------------------


Your prompt:  bye


Goodbye!


In [4]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."), HumanMessage(content='what about the Moon and Sun?'), AIMessage(content="The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system.")]
